In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!cp '/content/drive/MyDrive/Data/colabs_data/MOA_kaggle/quanvh8_funcs.py' .

In [3]:
'''ENSEMBLE NETS
Inspire by https://www.kaggle.com/c/otto-group-product-classification-challenge/discussion/14335'''

import numpy as np, pandas as pd, copy, tensorflow as tf, matplotlib.pyplot as plt

from tensorflow import feature_column as fc
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import (Dense, DenseFeatures, Dropout, 
                                     BatchNormalization, Embedding, Input, Concatenate, Average,
                                     InputLayer, Lambda)
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras import backend as K, Sequential, Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop

import tensorflow_addons as tfa
from tensorflow_addons.layers import WeightNormalization

from sklearn.preprocessing import QuantileTransformer, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans

from quanvh8_funcs import (DerivedFeatures, kfolds_bagging_training, voting_predict,
                           kolds_stacked_ensemble_training, stacked_ensemble_predict )

import sys

def log_loss_metric(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()
    return bce(y_true, y_pred).numpy()

print(pd.__version__)
print(tf.__version__)

1.1.4
2.3.0


# MODULE 1. DATA LOADING

In [4]:
# Loading data and encoding

folder_path = '/content/drive/My Drive/Data/colabs_data/MOA_kaggle/'
raw_test = pd.read_csv(folder_path + 'test_features.csv')
raw_train = pd.read_csv(folder_path + 'train_features.csv')
raw_targets = pd.read_csv(folder_path + 'train_targets_scored.csv')

# Phân loại dữ liệu
cols_id = ['sig_id']
cols_to_remove = ['cp_type']
cols_fts = [i for i in raw_train.columns if i not in cols_id +cols_to_remove]
cols_gene = [col for col in raw_train.columns if col.startswith("g-")]
cols_cell = [col for col in raw_train.columns if col.startswith("c-")]
cols_experiment = [col for col in cols_fts if col not in cols_gene+cols_cell]
cols_target = [i for i in raw_targets.columns if i not in cols_id]
num_fts, num_labels = len(cols_fts), len(cols_target)

# xử lý categorical
def transform_data(input_data):
    '''Clean data and encoding
        * input_data: table '''
    out = input_data.copy()
    out['cp_dose'] = out['cp_dose'].map({'D1':0, 'D2':1})
    out['cp_time'] = out['cp_time']/72
    
    return out

to_train = transform_data(raw_train[raw_train['cp_type'] != 'ctl_vehicle'])
to_train_targets = raw_targets.iloc[to_train.index]
full_pred  = transform_data(raw_test)
to_pred = full_pred[full_pred['cp_type'] != 'ctl_vehicle']

# MODULE 2. DATA TRANSFORMATION PIPELINE

In [5]:
# preprocessing pipeline
def pipe_line_builder(quantiles_num, pca_dims, kmean_clusters):
    '''Dựng pipe line cho từng nhóm columns
    :quantiles_num: int: số quantile khi normalise
    :pca_dims: int: số chiều pca'''
    norm = QuantileTransformer(n_quantiles=quantiles_num,random_state=0, output_distribution="normal")
    pca = PCA(n_components = pca_dims)
    derived_ft = DerivedFeatures(n_clusters = kmean_clusters)

    p_derived_ft = Pipeline([
        ('norm', norm), 
        ('derived', derived_ft)])

    p_norm_pca = Pipeline([ 
        ('norm', norm),
        ('pca', pca) ])
    return FeatureUnion([
        ('norm', norm), 
        ('norm_pca', p_norm_pca),
        ('derived', p_derived_ft)])

# 

pipe = Pipeline([
    ('norm_pca', ColumnTransformer([
                     ('gene', pipe_line_builder(quantiles_num = 200, pca_dims = 600, kmean_clusters = 5), cols_gene),
                     ('cell', pipe_line_builder(quantiles_num = 200, pca_dims = 50, kmean_clusters = 5), cols_cell),
                    ]) 
    ), 
    ('var', VarianceThreshold(0.5)) 
])

pipe = ColumnTransformer([
    ('gene_cell', pipe, cols_gene+ cols_cell),
    ('experiment', 'passthrough', cols_experiment)
])

In [6]:
# Transform data
pipe.fit(to_train[cols_fts].append(to_pred[cols_fts]))
X_train = pipe.transform(to_train[cols_fts])
X_pred = pipe.transform(to_pred[cols_fts])
y_train = to_train_targets[cols_target].values

In [7]:
print(X_train.shape, X_pred.shape, y_train.shape)
print(type(X_train), type(X_pred), type(y_train))

(21948, 1240) (3624, 1240) (21948, 206)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


# MODULE 3: BAGGING

In [8]:
# Hyper params
NFOLDS = 8
BATCH_SIZE = 128
EPOCHS = 150
BAGGING_ALPHA = 0.75
SEEDS = [23, 228, 1488, 1998, 2208, 2077, 404]
KFOLDS = 10
label_smoothing_alpha = 0.00005
P_MIN = label_smoothing_alpha
P_MAX = 1 - P_MIN

In [59]:
# Define model
model = Sequential([
    # BatchNormalization(),
    WeightNormalization(Dense(1024, activation="selu", kernel_regularizer= tf.keras.regularizers.l2(0.0005))),

    # BatchNormalization(),
    Dropout(0.25),
    WeightNormalization(Dense(1024, activation="selu", kernel_regularizer= tf.keras.regularizers.l2(0.0005))),
    # BatchNormalization(),
    Dropout(0.25),
    WeightNormalization(Dense(512, activation="selu", kernel_regularizer= tf.keras.regularizers.l2(0.0005))),
    # BatchNormalization(),
    Dropout(0.25),
    WeightNormalization(Dense(num_labels, activation="sigmoid"))
])

bce = tf.keras.losses.BinaryCrossentropy() #val_binary_crossentropy
model.compile(optimizer='adam', loss= BinaryCrossentropy(label_smoothing= label_smoothing_alpha)
              , metrics= [bce])

ValueError: ignored

In [57]:
# n_components = 256

# input_u = Input(shape= ( X_train.shape[1], ))
# layer_u = BatchNormalization() (input_u)
# layer_u = WeightNormalization(Dense(1024, activation="relu")) (layer_u)
# layer_u = BatchNormalization() (layer_u)
# layer_u = Dropout(0.25) (layer_u)
# layer_u = WeightNormalization(Dense(1024, activation="relu")) (layer_u)
# layer_u = BatchNormalization() (layer_u)
# layer_u = Dropout(0.25) (layer_u)

# layer_u = WeightNormalization(Dense(n_components, activation="relu")) (layer_u)
# layer_u = BatchNormalization() (layer_u)

# #2.1. Addition information for item_info
# chemical_category = tf.transpose(
#         tf.constant(
#             [[1 if '_inhibitor' in i else 0 for i in cols_target],
#                [1 if '_agonist' in i else 0 for i in cols_target],
#                [1 if '_agent' in i else 0 for i in cols_target],
#                [1 if '_antagonist' in i else 0 for i in cols_target],
#                [1 if '_blocker' in i else 0 for i in cols_target],
#                [1 if '_activator' in i else 0 for i in cols_target] 
#              ]))

# #2.2 Full item fts: addition + onehot
# item_ft = tf.concat(
#     [chemical_category ,
#      tf.eye(num_labels, dtype = tf.int32) # Create tensor 0-1 coresponse with chemical labels
#     ], axis = 1
# )
# layer_i = Dense(n_components, activation = 'relu', kernel_initializer='he_normal', name ='layer_u1') (item_ft)
# layer_i = BatchNormalization() (layer_i)
# #3. Dot product user - item
# def dot_2layer(x):
#     return K.dot( x[0], K.transpose(x[1]))
# dot_ui = Lambda( dot_2layer, name = 'lambda_dot' ) ([layer_u,layer_i])
# dot_ui = WeightNormalization(Dense(num_labels, activation = 'sigmoid', kernel_initializer='he_normal', name = 'labels'))(dot_ui)

# model = Model(inputs=[ input_u, ], outputs= [dot_ui])

# bce = tf.keras.losses.BinaryCrossentropy() #val_binary_crossentropy
# model.compile(optimizer='adam', loss= BinaryCrossentropy(label_smoothing= label_smoothing_alpha)
#               , metrics= [bce])

In [60]:
reduce_lr = ReduceLROnPlateau(monitor='val_binary_crossentropy', factor=0.3, patience=5, mode='min', min_lr=1E-5, verbose=1)
early_stopping = EarlyStopping(monitor='val_binary_crossentropy', min_delta=1E-5, patience=15, mode='min',restore_best_weights=True, verbose=1)
    
model.fit(
        X_train, y_train, validation_split = 0.3, 
        callbacks=[reduce_lr, early_stopping], epochs=150, verbose =1,
        batch_size=128)

Epoch 1/150
121/121 [==============================] - 1s 11ms/step - loss: 0.0134 - binary_crossentropy: 0.0134 - val_loss: 0.0181 - val_binary_crossentropy: 0.0178
Epoch 2/150
121/121 [==============================] - 1s 10ms/step - loss: 0.0133 - binary_crossentropy: 0.0133 - val_loss: 0.0181 - val_binary_crossentropy: 0.0178
Epoch 3/150
121/121 [==============================] - 1s 10ms/step - loss: 0.0133 - binary_crossentropy: 0.0132 - val_loss: 0.0181 - val_binary_crossentropy: 0.0178
Epoch 4/150
121/121 [==============================] - 1s 10ms/step - loss: 0.0133 - binary_crossentropy: 0.0132 - val_loss: 0.0181 - val_binary_crossentropy: 0.0179
Epoch 5/150
121/121 [==============================] - 1s 10ms/step - loss: 0.0133 - binary_crossentropy: 0.0131 - val_loss: 0.0181 - val_binary_crossentropy: 0.0179
Epoch 6/150
117/121 [============================>.] - ETA: 0s - loss: 0.0133 - binary_crossentropy: 0.0130
Epoch 00006: ReduceLROnPlateau reducing learning rate to 1e-05

In [ ]:
# model_2list = kfolds_training(NFOLDS, model, SEEDS, X_train, y_train, bagging_alpha = 0.8, bagging_samples = 15)
model_2list = kfolds_boosting_training(NFOLDS, model, SEEDS, X_train, y_train, n_interations = 5)

Training at fold:  0 ####################################################################################################
Epoch 1/150
151/151 [==============================] - 2s 15ms/step - loss: 0.0483 - mse: 0.0483 - val_loss: 0.0418 - val_mse: 0.0418
Epoch 2/150
151/151 [==============================] - 1s 10ms/step - loss: 0.0231 - mse: 0.0231 - val_loss: 0.0421 - val_mse: 0.0421
Epoch 3/150
151/151 [==============================] - 1s 10ms/step - loss: 0.0167 - mse: 0.0167 - val_loss: 0.0506 - val_mse: 0.0506
Epoch 4/150
151/151 [==============================] - 2s 10ms/step - loss: 0.0135 - mse: 0.0135 - val_loss: 0.0364 - val_mse: 0.0364
Epoch 5/150
151/151 [==============================] - 2s 10ms/step - loss: 0.0117 - mse: 0.0117 - val_loss: 0.0346 - val_mse: 0.0346
Epoch 6/150
151/151 [==============================] - 2s 10ms/step - loss: 0.0107 - mse: 0.0107 - val_loss: 0.0211 - val_mse: 0.0211
Epoch 7/150
151/151 [==============================] - 2s 10ms/step - loss

In [ ]:
# cols_target_low_score = to_train_targets[cols_target].sum().sort_values(ascending = True).head(2).index
# cols_target_low_score

In [ ]:
# nn_pred_train1 = voting_predict( sum(model_2list, []), X_train, 206)
# np.savetxt("NN_fts_pred_1.csv", nn_pred_train1, delimiter=",")
# single_model = model_2list[0][0]
# represent = tf.keras.models.Sequential(single_model.layers[:-1])
# nn_rept = represent.predict(X_train)
# np.savetxt("NN_fts_representation.csv", nn_rept, delimiter=",")

In [ ]:
prediction = voting_predict( sum(model_2list, []), X_pred, 206)

df_preds_non_ctl =  pd.DataFrame(prediction, columns= cols_target, index = to_pred.index)

# concat with all to pred values
df_preds = pd.concat([ full_pred[cols_id], df_preds_non_ctl], axis = 1).fillna(0)
# to csv
df_preds.to_csv("submission.csv", index = None)

In [ ]:
from google.colab import files
files.download("submission.csv") 

In [ ]:
# import glob, os

In [ ]:
# glob.glob('/content/drive/My Drive/Data/colabs_data/MOA_kaggle/Model_MOA/')

In [ ]:
# names = [os.path.basename(x) for x in glob.glob('/content/drive/My Drive/Data/colabs_data/MOA_kaggle/Model_MOA/')]

In [ ]:
# names